In [1]:
#!pip install twython

In [15]:
from twython import Twython
import pandas as pd
import numpy as np
import datetime as dt
import time
import yaml
import dask.dataframe as dd
from dask.multiprocessing import get

In [16]:
APP_KEY = 'OumzA854PAnWwA8VNjwZ6O1T1'
APP_SECRET = "aHWN7hVHFYGTJIqLG6d1sUgT2tzelmB7LNxtQjJz0RkmLDIFGW"

twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

In [17]:
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [18]:
locationsTemp=["chennai","delhi","india","ahmedabad","gujarat","tamil","jammu","srinagar","imphal","lucknow",
                "bengaluru","bangalore","bathinda","chandigarh","ludhiana","amritsar","punjab","arunachal","assam",
                "kolkata","westbengal","kerala","karnataka","andhra","telangana","hyderabad","madhya","uttar",
                "maharashtra","haryana",'Jammu','Srinagar','Delhi-NewDelhi','Bathinda','Dehradun','Chandigarh',
                'Ludhiana','Amritsar','Imphal','Lucknow','Jalandhar','Kolkata','Guwahati','Chennai','Patiala',
                'Bhubaneswar','Bengaluru','Patna','Jaipur','Coimbatore','Hyderabad','Ranchi','Thiruvananthapuram',
                'Shimla','Sangrur','Ahmedabad','Karnal','Pulwama','Puducherry','Gurgaon','Agartala','Madurai',
                'Tiruchirappalli','Bangkok','Salem']

locations = set()
for location in locationsTemp:
    locations.add(location.lower())
locations = pd.DataFrame(list(locations))
locations.columns = ["Locations"]
locations.to_csv("Locations of Interest.csv")

In [6]:
keywords = ['protest','protests','riot','riots','violence','unrest','clash','bandh','issue','rally','demonstration','election','polls','attac']
keywords = pd.DataFrame(keywords)
keywords.columns = ["Keywords"]
keywords.to_csv("Keywords.csv")

In [7]:
locations = pd.read_csv("Locations of Interest.csv", index_col=0)
keywords = pd.read_csv("Keywords.csv", index_col=0)

In [8]:
keywordsQuery = ''.join(map(str, [keyword+" OR " for keyword in list(keywords['Keywords'])]))[:-4]

In [9]:
start = 0
to_return = []
loc_list =locations["Locations"].values.tolist()
len_loc = len(loc_list) // 5 + 1
for i in range(5):
    to_return.append(loc_list[start:start+len_loc])
    start+=len_loc
    
locationQueries=[]
for row in to_return:
    locationQueries.append(''.join(map(str, [location+" OR " for location in row]))[:-4])
    

In [10]:
print("Locations Queries\n\n", locationQueries,"\n")
print("Keywords Query\n\n", keywordsQuery)

Locations Queries

 ['chandigarh OR delhi-newdelhi OR thiruvananthapuram OR shimla OR tiruchirappalli OR patna OR agartala OR bhubaneswar OR patiala OR bathinda OR bangkok', 'jaipur OR chennai OR tamil OR jammu OR karnataka OR ahmedabad OR andhra OR jalandhar OR uttar OR arunachal OR pulwama', 'guwahati OR india OR haryana OR dehradun OR bangalore OR kolkata OR telangana OR assam OR lucknow OR coimbatore OR karnal', 'salem OR delhi OR srinagar OR gujarat OR westbengal OR sangrur OR kerala OR ludhiana OR madhya OR amritsar OR hyderabad', 'gurgaon OR punjab OR bengaluru OR ranchi OR madurai OR maharashtra OR puducherry OR imphal'] 

Keywords Query

 protest OR protests OR riot OR riots OR violence OR unrest OR clash OR bandh OR issue OR rally OR demonstration OR election OR polls OR attac


In [11]:
date = dt.datetime.now()
cnt = 0
tweetList = []
for locationQuery in locationQueries:
    tweets = twitter.cursor(twitter.search, q=keywordsQuery + ' (' +locationQuery + ') -filter:retweets', count=100, tweet_mode = "extended")
    for item in tweets:
        if (cnt%10000 == 0):
            print("Progress: "+str(cnt))
        tweetList.append(item)
        cnt+=1
        if cnt > 40000:
            print("Thread sleep")
            time.sleep(60*15+5)
            cnt = 0
tweetsDF = pd.DataFrame(tweetList)
tweetsDF.to_csv("tweets_collected_"+str(date.day)+str(date.strftime("%m"))+".csv")

Progress: 0


C:\Users\Dan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: generator 'Twython.cursor' raised StopIteration
  


Progress: 10000
Progress: 20000
Progress: 30000
Progress: 40000
Thread sleep
Progress: 0
Progress: 10000


In [20]:
tweetsDF1 = pd.read_csv("tweets_collected_2704.csv", dtype={'id':object})
tweetsDF2 = pd.read_csv("tweets_collected_505.csv", dtype={'id':object})

C:\Users\Dan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,9,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Dan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
tweets = pd.concat([tweetsDF1, tweetsDF2])

In [19]:
cnt = 1
rowInd = 1
tempList = []
tweetList = []
tweetsDfs = []
date = dt.datetime.now()
for index, row in tweets.iterrows():
    tempList.append(row["id"])
    if cnt%100 == 0:
        try:
            tweetList.extend(twitter.lookup_status(id=tempList, tweet_mode = "extended"))
            tempList = []
        except:
            print("Succesful till: "+cnt)
    if (cnt%10000 == 0):
        print("Progress: "+str(cnt))
        if cnt >= 40000:
            tweetsDfs.append(pd.DataFrame(tweetList))
#             tweetsDf.to_csv("Tweets3Weeks"+str(date.day)+str(date.strftime("%m"))+"_"+str(rowInd)+".csv")
            tweetList = []
            rowInd+=1
#             print("Thread sleep")
#             time.sleep(60*15+5)
            cnt = 0
    cnt+=1
tweetList.extend(twitter.lookup_status(id=tempList, tweet_mode = "extended"))
tweetsDfs.append(pd.DataFrame(tweetList))

Progress: 10000
Progress: 20000
Progress: 30000
Progress: 40000
Progress: 10000
Progress: 20000
Progress: 30000
Progress: 40000


In [24]:
tweetsDF = pd.concat(tweetsDfs)

C:\Users\Dan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [36]:
tweetsDF.shape

(36298, 33)

In [38]:
tweetsDF.index = np.arange(len(tweetsDF))
tweetsDF.to_json('Tweets060519.json')

In [26]:
# from tqdm import tqdm_notebook as tqdm
def get_features(row):
    row["userlocation"] = row["user"]["location"]
    row["username"] = row["user"]["name"]
    row["userscreen_name"] = row["user"]["screen_name"]
    row["userdescription"] = row["user"]["description"]
    row["userfollowers_count"] = row["user"]["followers_count"]
    row["userfriends_count"] = row["user"]["friends_count"]
    row["userlisted_count"] = row["user"]["listed_count"]
    row["userfavourites_count"] = row["user"]["favourites_count"]
    row["userverified"] = row["user"]["verified"]
    row["userstatuses_count"] = row["user"]["statuses_count"]
    row["userfollowing"] = row["user"]["following"]
    row["userfollow_request_sent"] = row["user"]["follow_request_sent"]
    row["usercontributors_enabled"] = row["user"]["contributors_enabled"]
    sepr =  "" 
    l = []
    for location in locations['Locations']: 
        if (location in str(row["full_text"]).lower()):
            l.append(location)
    row["extracted_location"] = ','.join(l)
    return row

In [27]:
processed_data = tweetsDF.apply(get_features, axis=1)

In [29]:
processed_data.index = np.arange(len(processed_data))
processed_data.to_json('Tweets_latest.json')

In [ ]:
acledLocations = ['Jammu',
 'Srinagar',
 'Delhi-New Delhi',
 'Bathinda',
 'Dehradun',
 'Chandigarh',
 'Ludhiana',
 'Amritsar',
 'Imphal',
 'Lucknow',
 'Jalandhar',
 'Kolkata',
 'Guwahati',
 'Chennai',
 'Patiala',
 'Bhubaneswar',
 'Bengaluru',
 'Patna',
 'Jaipur',
 'Coimbatore',
 'Hyderabad',
 'Ranchi',
 'Thiruvananthapuram',
 'Shimla',
 'Sangrur',
 'Ahmedabad',
 'Karnal',
 'Pulwama',
 'Puducherry',
 'Gurgaon',
 'Agartala',
 'Madurai',
 'Tiruchirappalli',
 'Bangkok',
 'Salem','India']

In [16]:
def generate_locations(row):
    dfrow_list = []
    for location in acledLocations: 
        tempLocation = location
        tempLocation2 = location
        if (tempLocation == "Delhi-New Delhi"):
            tempLocation = "delhi"
        elif (tempLocation == "Bengaluru"):
            tempLocation = "bangalore"
            tempLocation2 = "karnataka"
        elif (tempLocation == "Kolkata"):
            tempLocation = "west bengal"
        elif (tempLocation == "Guwahati"):
            tempLocation = "assam"
        elif (tempLocation in ("Lucknow", "Dehradun")):
            tempLocation = "uttar"
        elif (tempLocation in ("Sangrur", "Ludhiana", "Amritsar", "Chandigarh", "Bathinda", "Jalandhar", "Patiala")):
            tempLocation = "punjab"
        elif (tempLocation in ("Karnal", "Gurgaon", "Chandigarh")):
            tempLocation2 = "haryana"
        elif (tempLocation == "Pulwama"):
            tempLocation2 = "jammu"
        elif (tempLocation == "Ahmedabad"):
            tempLocation = "gujarat"
        elif (tempLocation in ("Chennai", "Coimbatore", "Madurai", "Tiruchirappalli", "Salem")):
            tempLocation = "tamil"
        elif (tempLocation in ("Imphal", "Agartala")):
            tempLocation = "arunachal"
        elif (tempLocation == "Hyderabad"):
            tempLocation = "andhra"
            tempLocation2 = "telangana"
        elif (tempLocation == "Thiruvananthapuram"):
            tempLocation = "kerala"
        if (tempLocation.lower() in row["extracted_location"] or location.lower() in row["extracted_location"] or tempLocation2 in row["extracted_location"]):
            row["finallocation"] = location
            dfrow_list.append(row.copy())
#     print(row)
    return pd.DataFrame(dfrow_list)

In [ ]:
posssed_df = pd.concat(processed_data.apply(generate_locations, axis = 1).tolist())

In [ ]:
df = pd.read_json("../../../data/TweetsConsolidated.json")
df = pd.concat([posssed_df, df])
df.index = np.arange(len(df))
df.to_json("../../../data/TweetsConsolidated.json")